In [1]:
import numpy as np
import pandas as pd

## Read the `Wage` File

In [2]:
wage_df = pd.read_csv("wage.csv")

In [3]:
wage_df.head(3)

,year,age,maritl,race,education,region,jobclass,health,health_ins,logwage,wage
0,2006,18,1. Never Married,1. White,1. < HS Grad,2. Middle Atlantic,1. Industrial,1. <=Good,2. No,4.318063,75.043154
1,2004,24,1. Never Married,1. White,4. College Grad,2. Middle Atlantic,2. Information,2. >=Very Good,2. No,4.255273,70.476020
2,2003,45,2. Married,1. White,3. Some College,2. Middle Atlantic,1. Industrial,1. <=Good,1. Yes,4.875061,130.982177


In [4]:
wage_df.dtypes #we can see that everything except for year age (log) wage are categorical

year            int64
age             int64
maritl         object
race           object
education      object
region         object
jobclass       object
health         object
health_ins     object
logwage       float64
wage          float64
dtype: object

### Check unique values in the `jobclass` column

In [5]:
wage_df.jobclass.unique() #only two -- easy to replace

array(['1. Industrial', '2. Information'], dtype=object)

In [5]:
wage_df["job_information"] =  (wage_df["jobclass"] == "2. Information").astype(int) #so that =1 means information

In [6]:
wage_df.drop(['jobclass', 'logwage', 'region', 'year'], axis=1, inplace=True) #lets delete jobclass and logwage now

### Check unique values in the `health` column

In [7]:
wage_df.health.unique() #only two -- easy to replace

array(['1. <=Good', '2. >=Very Good'], dtype=object)

In [8]:
wage_df["health"] =  (wage_df["health"] == "2. >=Very Good").astype(int) #so that =1 means very good health

### Apply the same for `health_ins`

In [9]:
wage_df.health_ins.unique() #only two -- easy to replace

array(['2. No', '1. Yes'], dtype=object)

In [10]:
wage_df["health_ins"] =  (wage_df["health_ins"] == "1. Yes").astype(int) #so that =1 means has a health insurance

### Check unique values in the `maritl` column

In [11]:
wage_df.maritl.unique() #we cannot make this 1-2-3-4-5 as this is nominal

array(['1. Never Married', '2. Married', '4. Divorced', '3. Widowed',
       '5. Separated'], dtype=object)

In [12]:
one_hot = pd.get_dummies(wage_df.maritl, prefix='marriage')

In [13]:
wage_df = wage_df.join(one_hot)

In [14]:
wage_df.drop(['maritl', 'marriage_1. Never Married'], axis=1, inplace=True)

In [15]:
wage_df.columns = [*wage_df.columns[:-4], 'marriage_yes',\
                   'marriage_widowed', 'marriage_divorced', 'marriage_separated'] #we drop one -- why?

### Decide what to do for `education`

In [16]:
wage_df.education.unique() #it looks like we can take these as ordinal categories

array(['1. < HS Grad', '4. College Grad', '3. Some College', '2. HS Grad',
       '5. Advanced Degree'], dtype=object)

In [17]:
wage_df.education = (wage_df.education.astype(str).str[0]).astype(int)

### Question to deliver on encoding categorical variables
- Encode the `race` variable by using numerical variable(s) and drop `race`.
- Now that all the variables in our dataframe is numerical, apply a linear regression model to explain the `wage` with the other variables. Report the coefficients.
- Interpret the coefficient of the `education` varaible.

#### First encode the variable. Then fit a linear model.

In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
wage_df.race.unique()

array(['1. White', '3. Asian', '4. Other', '2. Black'], dtype=object)

In [21]:
one_race_hot = (pd.get_dummies(wage_df.race, prefix='race'))

In [22]:
wage_df = wage_df.join(one_race_hot)

In [23]:
wage_df.drop(['race', 'race_4. Other'], axis=1, inplace=True)

In [24]:
wage = wage_df['wage']

In [26]:
wage_df.drop('wage', axis=1, inplace=True)

In [27]:
reg = LinearRegression().fit(wage_df, wage) #hint: to get coefficients of the fitted model use "reg.coef_"

In [29]:
reg.coef_

array([ 0.3037476 , 13.34408106,  6.60422963, 16.99604481,  3.78564598,
       17.17994727,  0.97270744,  3.49189903, 12.13254899,  4.73430099,
       -0.40801293,  3.24613976])

In [30]:
wage_df

,age,education,health,health_ins,job_information,marriage_yes,marriage_widowed,marriage_divorced,marriage_separated,race_1. White,race_2. Black,race_3. Asian
0,18,1,0,0,0,0,0,0,0,1,0,0
1,24,4,1,0,1,0,0,0,0,1,0,0
2,45,3,0,1,0,1,0,0,0,1,0,0
3,43,4,1,1,1,1,0,0,0,0,0,1
4,50,2,0,1,1,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,44,3,1,1,0,1,0,0,0,1,0,0
2996,30,2,1,0,0,1,0,0,0,1,0,0
2997,27,1,0,0,0,1,0,0,0,0,1,0
2998,27,3,1,1,0,0,0,0,0,1,0,0


Education has a coefficient of 13.3. This is the 3rd highest coefficient and as such has the 3rd highest positive correlation with wage, after marriage and health insurance.